## Globals

In [61]:
import os
import glob
import subprocess
import time
import mmap


rootdir = '/home/preclineu/ramcir/Desktop/Diffusion/'
qcdir = os.path.join(rootdir, 'QC_dmri')

if os.path.exists(qcdir) == False:
    os.mkdir(os.path.join(rootdir, 'QC_dmri'))
else: pass

cluster_log_dir = '/home/preclineu/ramcir/DCCN/TorqueLogs'

cmd_qsub_base = ['/home/preclineu/ramcir/DCCN/Scripts/SubmitToCluster.py',
                 '-length', '102400',
                 '-memory', '2gb',
                 '-logfiledir', cluster_log_dir
                ]

# get a list of subjects to process
sub_dirs = sorted(glob.glob(os.path.join(rootdir,'subjects','*')))
print('Found', len(sub_dirs), 'subjects to process')

Found 20 subjects to process


### Prepare subjects for QC by creating missing files

In [62]:
def prep_sub(dmridir):
        
    mask_dir = os.path.join(dmridir,'nodif_mask.nii.gz')
    acqparam_file = os.path.join(dmridir, 'acqparam.txt')
    idx_file = os.path.join(dmridir, 'index.txt')
    
    #Create symbolic link for data_ud with the name data
    #This is necessary for running 'quad' since all other eddy outputs are called data.eddy.-
    data_ud = os.path.join(dmridir, 'data_ud.nii.gz')
    data = os.path.join(dmridir, 'data.nii.gz')
    
    if os.path.islink(data) == False:
        os.symlink(data_ud, data)
    else:
        pass

    #Write command for creating the b0 mask and run the bash process
    if os.path.exists(mask_dir) == False:
        fslroi_cmd = ['fslroi ' + dmridir + '/data.nii.gz',
              dmridir + '/b0_data.nii.gz' ' 0 1']
        fslbet_cmd = ['bet ' + dmridir + '/b0_data.nii.gz',
              dmridir + '/nodif' ' -f 0.1 -g 0 -m -n']

        cmd_str = '"%s"' % str(' '.join(fslroi_cmd) + ' && ' + ' '.join(fslbet_cmd) ) 
        cmd_qsub = cmd_qsub_base + ['-name', 'UKB_fsl_roi','-command', cmd_str]
        subprocess.run(' '.join(cmd_qsub), shell=True)       
    else:
        pass

    #Create aquision parameters file and index file
    #The acquisition parameters file is based on dataset acq documentation
    #i.e. AP dir = 0 1 0, PA dir = 0 -1 0, Echo spacing = 0.67 ms, EPI factor = 104 (for UKB)
    if os.path.exists(acqparam_file) == False:
        with open(os.path.join(dmridir,'acqparams.txt'), 'w') as acqparams:
            acqparams.write('0 1 0 0.069\n0 -1 0 0.069')
    else:
        pass

    #The index file indicated the direction in which each volume was aquired (based on aqcparams.txt)
    #count number of volumes present in data
    if os.path.exists(idx_file) == False:
        with open(os.path.join(dmridir,'bvals'), 'r') as bvals:
            data = bvals.read()
            vols = data.split()
            no_vols = len(vols)
        with open(os.path.join(dmridir,'index.txt'), 'w') as index:
            for vol in range(0,no_vols):    
                index.write('1 ')
    else:
        pass

## Run QUAD
    1. Check for missing files
    2. Create missing files (with prep_sub)
    3. Run QUAD

In [68]:
num_submitted = 0
num_completed = 0
sub_no_dmri = []
sub_no_eddy = []
sub_ready = []

# Control the number of jobs to submit
sdirs = sub_dirs[0:3]
quad_folders = open(os.path.join(qcdir,'quad_folders.txt'), 'a')

for s in sdirs:
    subid = os.path.splitext(os.path.basename(s))[0]

    dmridir = os.path.join(s, 'dMRI', 'dMRI')
    quaddir = os.path.join(dmridir, 'data.qc')  
    eddyfile = os.path.join(dmridir, 'data.eddy_parameters')

    #First check if dMRI data is avalable
    if os.path.exists(dmridir) == False:  
        print(subid,' dMRI data not found. Doing nothing')
        sub_no_dmri.append(subid)
    else: 
        #Check if EDDY files are available
        if os.path.exists(eddyfile) == False:
            print(subid,' EDDY data not found. Doing nothing')
            sub_no_eddy.append(subid)
        else:  
            #Check if quad has already been run
            if os.path.exists(quaddir) == True:
                print(subid,' QC already done. Doing nothing')
                
                #check if sub is on the quad list of paths
                with open(os.path.join(qcdir,'quad_folders.txt'), 'r') as f:
#                     s = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
                    if quaddir in f.read():
                        continue
                    else:
                        quad_folders.write(quaddir + '\n') 
                num_completed += 1
            else:
                print(subid,' is being prepared for QC')
                prep_sub(dmridir)
                
                time.sleep(20)
        
                print(subid,' is running QUAD')       
                quad_cmd = ['eddy_quad ' + dmridir + '/data',
                            '-idx ' + dmridir + '/index.txt',
                            '-par ' + dmridir + '/acqparams.txt',
                            '-m ' + dmridir + '/nodif_mask', 
                            '-b ' + dmridir + '/bvals ' 
                            '-g '  + dmridir + '/bvecs']

                cmd_str = '"%s"' % str(' '.join(quad_cmd)) 
                cmd_qsub = cmd_qsub_base + ['-name', 'UKB_eddy_quad','-command', cmd_str]
                subprocess.Popen(' '.join(cmd_qsub), shell=True)
                quad_folders.write(quaddir + '\n')
                num_submitted += 1        
                
                time.sleep(30)
                
                #Check if quad folder was created
                if os.path.exists(quaddir) == True:
                    print(subid,' QC done!')
                    quad_folders.write(quaddir + '\n')
                    num_completed += 1
                else:
                    print(subid,' QC failed.')
        
quad_folders.close()

1000050  EDDY data not found. Doing nothing
1000432  is being prepared for QC
1000432  is running QUAD
1000432  QC failed.
1000853  is being prepared for QC
1000853  is running QUAD
1000853  QC failed.


In [59]:
def squad_sub(qcdir):    
        squad_cmd = 'eddy_squad'
        quad_folders = os.path.join(qcdir,'quad_folders.txt')
        prep_cmd = squad_cmd + ' ' + os.path.join(qcdir,'quad_folders.txt') + ' -o ' + os.path.join(qcdir,'squad')
        subprocess.run(prep_cmd, shell = True)        

In [60]:
squad_sub(qcdir)